<a href="https://colab.research.google.com/github/Jatish-Khanna/AISamples/blob/main/Day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import openai
from datetime import datetime
from advanced_context_manager import AdvancedContextManager, ConversationMemory, ContextStrategy

# Page config
st.set_page_config(page_title="Advanced AI Assistant", layout="wide")

# Initialize
if "context_manager" not in st.session_state:
    st.session_state.context_manager = AdvancedContextManager(max_tokens=4000)

if "memory" not in st.session_state:
    st.session_state.memory = ConversationMemory()

# Sidebar controls
with st.sidebar:
    st.header("Advanced Context Controls")

    # Context strategy selection
    strategy_options = {
        "Sliding Window": ContextStrategy.SLIDING_WINDOW,
        "Summarization": ContextStrategy.SUMMARIZATION,
        "Importance-Based": ContextStrategy.SEMANTIC_COMPRESSION,
        "Hierarchical": ContextStrategy.HIERARCHICAL
    }

    selected_strategy = st.selectbox(
        "Context Strategy",
        options=list(strategy_options.keys()),
        index=0
    )

    # Update strategy if changed
    new_strategy = strategy_options[selected_strategy]
    if st.session_state.memory.context_strategy != new_strategy:
        st.session_state.memory.context_strategy = new_strategy
        st.session_state.context_manager.strategy = new_strategy

    # Token limit adjustment
    max_tokens = st.slider(
        "Max Context Tokens",
        min_value=1000,
        max_value=8000,
        value=4000,
        step=500
    )
    st.session_state.context_manager.max_tokens = max_tokens

    st.divider()

    # Context statistics
    stats = st.session_state.context_manager.get_context_stats(st.session_state.memory)

    st.subheader("Context Analytics")

    col1, col2 = st.columns(2)
    with col1:
        st.metric("Messages", stats.get("total_messages", 0))
        st.metric("Token Usage", f"{stats.get('token_utilization', 0):.1%}")

    with col2:
        st.metric("Total Tokens", stats.get("total_tokens", 0))
        st.metric("Avg Importance", f"{stats.get('avg_importance', 0):.1f}")

    # Message type breakdown
    if stats.get("message_types"):
        st.subheader("Message Types")
        for msg_type, count in stats["message_types"].items():
            st.text(f"{msg_type}: {count}")

    # Session info
    st.subheader("Session Info")
    st.text(f"Strategy: {stats.get('strategy', 'unknown')}")
    st.text(f"Duration: {stats.get('session_duration', 0):.1f} min")
    st.text(f"Summaries: {stats.get('summaries_created', 0)}")

    st.divider()

    # Advanced controls
    st.subheader("Advanced Controls")

    if st.button("Clear Conversation"):
        st.session_state.memory = ConversationMemory(
            context_strategy=new_strategy
        )
        st.rerun()

    if st.button("Force Summarization"):
        # Temporarily lower threshold to force summarization
        original_threshold = st.session_state.context_manager.summary_threshold
        st.session_state.context_manager.summary_threshold = 0
        st.session_state.memory = st.session_state.context_manager._summarization_strategy(
            st.session_state.memory
        )
        st.session_state.context_manager.summary_threshold = original_threshold
        st.rerun()

    if st.button("Export Conversation"):
        export_data = st.session_state.context_manager.export_conversation(
            st.session_state.memory
        )
        st.download_button(
            "Download JSON",
            data=export_data,
            file_name=f"conversation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
            mime="application/json"
        )

# Main interface
st.title(f"🧠 Advanced AI Assistant")
st.caption(f"Context Strategy: {selected_strategy} | Token Limit: {max_tokens:,}")

# Display token usage bar
if stats.get("total_tokens", 0) > 0:
    usage_pct = stats["token_utilization"]
    st.progress(usage_pct)

    if usage_pct > 0.8:
        st.warning("Context approaching token limit - older messages may be compressed")
    elif usage_pct > 0.9:
        st.error("Context nearly full - active memory management in effect")

# Chat interface with enhanced display
chat_container = st.container()

with chat_container:
    for i, (message, metadata) in enumerate(zip(st.session_state.memory.messages, st.session_state.memory.metadata)):
        role = message.get("role", "")
        content = message.get("content", "")

        # Enhanced message display with metadata
        if role == "system":
            if "[SUMMARY]" in content or "[ARCHIVED]" in content:
                with st.expander(f"📋 {metadata.message_type.title()} (Importance: {metadata.importance_score:.1f})"):
                    st.info(content)
            else:
                st.info(content)
        elif role == "user":
            with st.chat_message("user"):
                st.write(content)
                # Show metadata in debug mode
                if st.checkbox(f"Show metadata {i}", key=f"meta_{i}"):
                    st.json({
                        "timestamp": metadata.timestamp.strftime("%H:%M:%S"),
                        "tokens": metadata.token_count,
                        "importance": metadata.importance_score,
                        "type": metadata.message_type
                    })
        elif role == "assistant":
            with st.chat_message("assistant"):
                st.write(content)

# Input area
user_input = st.chat_input("Ask me anything...")

if user_input:
    # Add user message
    user_message = {"role": "user", "content": user_input}
    st.session_state.memory = st.session_state.context_manager.add_message(
        st.session_state.memory,
        user_message
    )

    # Display user message
    with st.chat_message("user"):
        st.write(user_input)

    # Generate response
    with st.chat_message("assistant"):
        try:
            # Prepare messages for API
            api_messages = [{"role": "system", "content": "You are a helpful assistant."}]

            # Add conversation messages
            for msg in st.session_state.memory.messages:
                if msg.get("role") in ["user", "assistant"]:
                    api_messages.append(msg)

            # Call OpenAI API
            with st.spinner("Thinking..."):
                response = openai.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=api_messages[-10:],  # Last 10 messages to avoid context overflow
                    temperature=0.7,
                    max_tokens=1000
                )

                assistant_response = response.choices[0].message.content
                st.write(assistant_response)

                # Add assistant response to memory
                assistant_message = {"role": "assistant", "content": assistant_response}
                st.session_state.memory = st.session_state.context_manager.add_message(
                    st.session_state.memory,
                    assistant_message
                )

        except Exception as e:
            st.error(f"Error: {e}")

# Context visualization
st.divider()
with st.expander("🔍 Context Visualization"):
    if st.session_state.memory.messages:
        st.subheader("Message Importance Timeline")

        # Create a simple visualization of message importance over time
        importance_data = []
        for i, (msg, meta) in enumerate(zip(st.session_state.memory.messages, st.session_state.memory.metadata)):
            importance_data.append({
                "Message": i + 1,
                "Importance": meta.importance_score,
                "Tokens": meta.token_count,
                "Role": msg.get("role", "unknown"),
                "Type": meta.message_type
            })

        st.dataframe(importance_data, use_container_width=True)

        # Show memory management events
        st.subheader("Memory Management Log")
        if st.session_state.memory.summaries:
            st.write(f"📝 {len(st.session_state.memory.summaries)} summarization events")
            for i, summary in enumerate(st.session_state.memory.summaries):
                st.text(f"Summary {i+1}: {summary['content'][:100]}...")
        else:
            st.write("No memory management events yet")

# Testing panel
st.divider()
with st.expander("🧪 Strategy Testing"):
    st.subheader("Test Context Strategies")

    col1, col2 = st.columns(2)

    with col1:
        if st.button("Add Spam Messages"):
            spam_messages = [
                {"role": "user", "content": f"Test message {i}"},
                {"role": "assistant", "content": f"Response {i}"}
                for i in range(5)
            ]

            for msg in spam_messages:
                st.session_state.memory = st.session_state.context_manager.add_message(
                    st.session_state.memory, msg
                )
            st.rerun()

    with col2:
        if st.button("Add Important Message"):
            important_msg = {
                "role": "user",
                "content": "This is extremely important information that I need to remember for later: The secret code is 12345."
            }
            st.session_state.memory = st.session_state.context_manager.add_message(
                st.session_state.memory, important_msg
            )
            st.rerun()

# Footer
st.divider()
st.caption("Day 3: Advanced Context Management - Understanding how AI systems handle memory constraints")